# Ca II 8542 inversion

In [71]:
from __future__ import print_function
%matplotlib tk
from ReductionImports import *
import  NicoleNative as nic
setrcParams()
# dir_init = getFileFilesDir(2)
dir_init = '/mnt/Data/CaII Polarimeter v2/Level-2'
nicole_dir = '/home/hp/Dropbox/Inversions/NICOLE.git/run'
print(dir_init)

/mnt/Data/CaII Polarimeter v2/Level-2


In [ ]:
# Done so far (chi-square)
# 57, 55, 1, 1 (1.1, 0.8)
# 54, 50, 1, 1 (1.2, 1.2)
# 56, 54
# 55, 55, 4, 4 ()

# Load Stokes data file

In [3]:
stokes_file = getFileFilesDir(0, initialdir = dir_init, 
                                 filetypes  = (('FITS file', '*STOKES*.fits'),), 
                                 title = 'Choose the stack of Stokes files...') # Select the file
hdu = pf.open(stokes_file) # Open with fits package
stokes_obs = float32(hdu[0].data) # Read the data
header = hdu[0].header # Read the header
hdu.close() # Clos HDU
print(stokes_file, ' is loaded')
ns, nw, ny, nx = stokes_obs.shape
#---------------------------------------------------- Fixed Parameters

/mnt/Data/CaII Polarimeter v2/Level-2/STOKES_ZIMPOL3.fits  is loaded


In [25]:
pix_caii = 603
pix_fei = 294
pix_sii = 157
wav_caii = 8542.089
wav_atm1 = 8540.8
wav_fei = 8538.015
wav_sii = 8536.165
plate_scale = 0.1
lin_disp = 0.0132
nw_ran = (arange(nw) - pix_caii)*lin_disp + wav_caii

In [26]:
stokes = np.copy(stokes_obs)
stokes[1::] /= stokes[0]

In [27]:
previewImage(stokes[0,100,:,:])

In [28]:
previewPlot(stokes[3,:,75,12])
axhline(0)

# Select RoI for profiles

In [82]:
x_bin, y_bin = 4,4
x_beg, x_end, y_beg, y_end = get_xy(stokes_obs[0,0], x_bin, y_bin)
print(x_beg, x_end, y_beg, y_end)
# Relevant dir and names
path_dirs = os.path.normpath(stokes_file).split(SEP)
savedir = SEP.join(path_dirs[0:-2]) + SEP + 'Level-3' + SEP + 'ZIMPOL3' + SEP
if not os.path.exists(savedir):
    os.makedirs(savedir)
suffix = '_'.join(['CaII', str(x_beg), str(y_beg), str(x_bin), str(y_bin)])
print(savedir, suffix)
model_guess = savedir + 'modguess_' + suffix + '.model'
model_fit = savedir + 'modfit_' + suffix + '.mod'
model_fit2 = savedir + 'modfit_' + suffix + '.mod2'
model_fit3 = savedir + 'modfit_' + suffix + '.mod3'
profile_obs = savedir + 'proobs_' + suffix + '.pro'
profile_fit = savedir + 'profit_' + suffix + '.pro'
profile_fit2 = savedir + 'profit_' + suffix + '.pro2'
profile_fit3= savedir + 'profit_' + suffix + '.pro3'
# stokes_disp = mean(stokes_obs[:,:, y_beg:y_end, x_beg:x_end], axis=(2,3))
# nic.plot_profile(figure(0), [stokes_disp])

4 8 71 75
/mnt/Data/CaII Polarimeter v2/Level-3/ZIMPOL3/ CaII_4_71_4_4


# Create Profiles

In [83]:
win_smooth = 0.001
wav_off = 0
wav_n = 180
pix_start = int(pix_caii-wav_n/2)
# wav_n = 800
# pix_start = 0
wav_ran = nw_ran[pix_start:pix_start+wav_n] + wav_off
stokes_temp = mean(stokes_obs[:,pix_start:pix_start+wav_n, y_beg:y_end, x_beg:x_end], axis=(2,3))
stokes_temp = gaussian_filter1d(stokes_temp, win_smooth)
stokes_temp /= 100000
stokes_temp[1] += 0.000*stokes_temp[3]
stokes_temp[2] += 0.000*stokes_temp[3]
stokes_temp[3,:] = 1.2*stokes_temp[3,:]
nic.plot_profile(figure(0), [stokes_temp])
observ = transpose(append(reshape(wav_ran, (1, wav_n)) , stokes_temp, axis=0))
savetxt(profile_obs, observ, fmt='{:^10}'.format('%s'), delimiter=' ')

In [84]:
weights = np.copy(observ)
weights[:,1::] = 10**10
weights[40:-20, 1] = 0.002
weights[:, 2] = 0.01
weights[:, 3] = 0.01
weights[:,4] = 0.002
savetxt('Weights.pro', weights, fmt='{:^10}'.format('%s'), delimiter=' ')

In [85]:
try:
    os.remove('Weights.pro')
except:pass

# Create guess model

In [99]:
# New model
standard_model = nicole_dir + SEP + 'falc.model' 
me_pars, me_head, me_const, me_out = nic.read_ascii_model(standard_model)
me_const = ' ' + str(int((lin_disp*2.0/8538)*2.998*10**10)) + ' 0.0\r\n'
N = me_out.shape[0]
# tau = np.copy(me_out[:,0])
# tau -= tau.min()
# tau /= tau.max()
# me_out[:,5] = 200000*tau  # v_los
# me_out[:,4] = -1300*tau  # b_los
# me_out[:,6] = -100*tau  # b_x
# me_out[:,7] = 400*tau  # b_y
nic.create_ascii_model(model_guess, me_head, me_const, me_out)
mi_pars, mi_head, mi_const, mi_plot = nic.read_ascii_model(model_guess)
fig = figure(None, figsize=(10,6))
nic.plot_model_bvtp(fig, [mi_plot])

# Invert

In [100]:
try:
    os.remove(profile_fit)
    os.remove(model_fit)
except: pass
nicole_input = nicole_dir + SEP + 'NICOLE.input'
INPUT = nic.set_nicole_input_params(Mode = 'I',
                                    Printout__detail = 1,
                                    Input__model = model_guess,
                                    Observed__profiles = profile_obs,
                                    Output__profiles = profile_fit,
                                    Heliocentric__angle = 0.97, 
                                    Output__model = model_fit,
                                    noise__level = 0.0005,
                                    Maximum__number__of__inversions=5,
                                    Continuum__reference = 3,
                                    Debug__mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                     First__wavelength = wav_ran[0], 
                                     Wavelength__step = lin_disp,
                                     Number__of__wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                     Line = 'CaII__8542',
                                     Mode = 'NLTE')
INPUT = nic.set_nicole_regions_lines(INPUT, 'Nodes',
                                     Temperature = 5,
                                     Velocity = 2,
                                     Microturbulence = 1,
                                     Bz = 2,
                                     By = 2,
                                     Bx = 2)
INPUT = nic.set_nicole_regions_lines(INPUT, 'NLTE',
                                     Linear__formal__solution = 1)

F = open(nicole_input, 'w')
F.write(INPUT)
F.close()

# Plot model, profiles

In [102]:
s_out = nic.read_native_profile([profile_fit])
fig = figure(None, figsize=(10,6))
nic.plot_profile(fig, [stokes_temp], color='k', ls='-.', lw=0.6)
# nic.plot_profile(fig, [gaussian_filter1d(stokes_temp, 4)], color='k', ls='--', lw=0.8)
nic.plot_profile(fig, s_out, color='k', ls='-', lw=1)
fig.tight_layout()
savefig(savedir+'profit_'+suffix+'.eps', format='eps', dpi=1200)

/mnt/Data/CaII Polarimeter v2/Level-3/ZIMPOL3/profit_CaII_4_71_4_4.pro 
 b'nicole2.3bp     '
1 1 180


/home/hp/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [103]:
mo_pars, mo_head, mo_plot, mo_const, mo_abu = nic.read_native_model(model_fit)
fig = figure(None, figsize=(10,6))
nic.plot_model_bvtp(fig, [mi_plot, mo_plot])
fig.tight_layout()
savefig(savedir+'modfit_'+suffix+'.eps', format='eps', dpi=1200)

# Invert again

In [55]:
try:
    os.remove(profile_fit2)
    os.remove(model_fit2)
except: pass
INPUT = nic.set_nicole_input_params(Mode = 'I',
                                    Printout__detail = 1,
                                    Input__model = model_fit,
                                    Observed__profiles = profile_obs,
                                    Output__profiles = profile_fit2,
                                    Heliocentric__angle = 0.97, 
                                    Output__model = model_fit2,
                                    noise__level = 0.005,
                                    Maximum__number__of__inversions=5,
                                    Continuum__reference = 3,
                                    Debug__mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                     First__wavelength = wav_ran[0], 
                                     Wavelength__step = lin_disp,
                                     Number__of__wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                     Line = 'CaII__8542',
                                     Mode = 'NLTE')
INPUT = nic.set_nicole_regions_lines(INPUT, 'Nodes',
                                     Temperature = 8,
                                     Velocity = 3,
                                     Microturbulence = 1,
                                     Bz = 3,
                                     By = 3,
                                     Bx = 3)

F = open('NICOLE.input', 'w')
F.write(INPUT)
F.close()

# Plot again

In [58]:
s_out2 = nic.read_native_profile([profile_fit2])
# s_out2 = [gaussian_filter1d(s_out[0], 3, axis=1)]
fig = figure(None, figsize=(10,6))
nic.plot_profile(fig, [stokes_temp], color='k', ls='-.', lw=0.5)
nic.plot_profile(fig, s_out, color='k', ls='--', lw=1)
nic.plot_profile(fig, s_out2, color='k', ls='-', lw=0.8)
fig.tight_layout()
savefig(savedir+'profit2_'+suffix+'.eps', format='eps', dpi=1200)

./20180426_085155/profit_CaII_52_54_49_51.pro2 
 b'nicole2.3bp     '
1 1 180


/home/hp/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [59]:
mo_pars2, mo_head2, mo_plot2, mo_const2, mo_abu2 = nic.read_native_model(model_fit2)
fig = figure(None, figsize=(10,6))
nic.plot_model_bvtp(fig, [mi_plot, mo_plot, mo_plot2])
fig.tight_layout()
savefig(savedir+'modfit2_'+suffix+'.eps', format='eps', dpi=1200)